In [1]:
import import_ipynb #allows access to import other notebooks
from PreprocessLib import cleanText, removeSingleLists

importing Jupyter notebook from PreprocessLib.ipynb


In [2]:
#processes files for Doc2Vec
def process_file_doc(a_file):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks

    cleanedFile = cleanText(sentences, False) #cleanText is found in PreprocessLib.ipynb
    return cleanedFile

In [4]:
import spacy
#processes files for POS tags
def process_file(a_file, tokenize=True):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks
    
    
   
    if (tokenize == False):
        cleanedFile = cleanText(sentences, tokenize)
        return cleanedFile
    cleanedFile = cleanText(sentences) #cleanText is found in PreprocessLib.ipynb
        
    output_doc = []
    nlp = spacy.load("el_core_news_sm")

    for sentence in cleanedFile:
        temp = []
        doc = nlp(sentence)
        for token in doc:
            temp.append(token.text)
        output_doc.append(temp)
        output_doc = removeSingleLists(output_doc)
    return output_doc

In [5]:
longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")
#processes files for D2V
long = process_file_doc(longFile)
mark = process_file_doc(markFile)
math = process_file_doc(mathFile)
ignat = process_file_doc(ignatFile)
test = process_file_doc(testFile)
#processes files for POS tag
longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")
longFreq = process_file(longFile, False)
markFreq = process_file(markFile, False)
mathFreq = process_file(mathFile, False)
ignatFreq = process_file(ignatFile, False)
testFreq = process_file(testFile, False)

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#places the texts in a list with tags for each author
docs = []
docs.append(TaggedDocument(words=long, tags=["Longus"]))
docs.append(TaggedDocument(words=mark, tags=["Mark"]))
docs.append(TaggedDocument(words=math, tags=["Mathew"]))
docs.append(TaggedDocument(words=ignat, tags=["Ignatius"]))

In [5]:
import gensim
model = gensim.models.doc2vec.Doc2Vec(vector_size = 5)
model.build_vocab(docs)
model.train(docs, total_examples=model.corpus_count, epochs=300)

In [41]:
model.save("doc2vec.model")

In [6]:
import numpy as np
trainX = np.array([model.docvecs[0], model.docvecs[1], model.docvecs[2], model.docvecs[3]])
trainY = ["Longus", "Mark", "Mathew", "Ignatius"]
docVecs = {}
docVecs['Test'] = model.infer_vector(test.split())
testX = np.array(docVecs["Test"])

In [39]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=4000, hidden_layer_sizes=(4,)).fit(trainX, trainY)

In [40]:
#results for using the entire document as a vector
print(clf.predict_proba(testX.reshape(1, -1)))
print(clf.predict(testX.reshape(1, -1)))

[[9.9999833e-01 1.6844411e-06 3.6473325e-17 3.1051687e-08]]
['Ignatius']


In [33]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def sliceCorpus(txt, label, docs):
    for sentence in txt:
        docs.append(TaggedDocument(words=sentence, tags=[label]))

In [34]:
docs = []
sliceCorpus(long, "Longus", docs)
sliceCorpus(mark, "Mark", docs)
sliceCorpus(math, "Mathew", docs)
sliceCorpus(ignat, "Ignatius", docs)

In [17]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#takes in a text "txt", a label "label", D2V docs, and a number of sentences "sentenceNum"
#returns the text in split-up format of sentenceNum sentences.
def sliceCorpustoPara(txt, label, docs, sentenceNum):
    tmpList = ""
    finalList = []
    j = 0
    for i in range(len(txt)):
        tmpList += txt[i] + " "
        if (j == sentenceNum):
            finalList.append(tmpList)
            docs.append(TaggedDocument(words=tmpList, tags=[label]))
            tmpList = ""
            j = 0
        j += 1
    if len(tmpList):
        docs.append(TaggedDocument(words=tmpList, tags=[label]))
        finalList.append(tmpList)
    return finalList

In [186]:
#prepares text for paragraph level D2V
docs=[]
sliceCorpustoPara(longSent, 'Longus', docs, 3)
sliceCorpustoPara(markSent, 'Mark', docs, 3)
sliceCorpustoPara(mathSent, 'Mathew', docs, 3)
sliceCorpustoPara(ignatSent, 'Ignatius', docs, 3)
tmp = []
testPara = sliceCorpustoPara(testSent, "Test", tmp, 3)

In [64]:
longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")
#sentenec tokenizes the texts
long = stringify(longFile)
mark = stringify(markFile)
math = stringify(mathFile)
ignat = stringify(ignatFile)
test = stringify(testFile)
from nltk import tokenize
longSent = tokenize.sent_tokenize(long)
markSent = tokenize.sent_tokenize(mark)
mathSent = tokenize.sent_tokenize(math)
ignatSent = tokenize.sent_tokenize(ignat)
testSent = tokenize.sent_tokenize(test)

In [63]:
#returns the file as a long string
def stringify(a_file):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks
    return sentences

In [74]:
import numpy as np
import gensim
from sklearn.neural_network import MLPClassifier
#for each iteration, prints results of logisitic regression classifier,
#   MLPClassifer and linerSVC classifier
#Keeps track of the average result of each iteration per author and prints at the end
def runMLAnalysis():
    avCl1 = 0
    avLu1 = 0
    avPa1 = 0
    avPu1 = 0
    avCl2 = 0
    avLu2 = 0
    avPa2 = 0
    avPu2 = 0
    iterations = 20
    svcCount = 0
    for i in range(iterations):

        
        model = gensim.models.doc2vec.Doc2Vec()
        model.build_vocab(docs)
        model.train(docs, total_examples=model.corpus_count, epochs=300)
        trainX = np.array([model.docvecs[0], model.docvecs[1], model.docvecs[2], model.docvecs[3]])
        trainY = ["Longus", "Mark", "Mathew", "Ignatius"]
        docVecs = {}
        docVecs['Test'] = model.infer_vector(test.split())
        testX = np.array(docVecs["Test"])
        clf = MLPClassifier(max_iter=8000, hidden_layer_sizes=(4,)).fit(trainX, trainY)
        x = clf.predict_proba(testX.reshape(1, -1))
        print("MLP")
        print(clf.classes_)
        print(x)
        avCl1 += x[0][0]
        avLu1 += x[0][1]
        avPa1 += x[0][2]
        avPu1 += x[0][3]
        from sklearn.linear_model import LogisticRegression
        clf = LogisticRegression()
        clf.fit(trainX, trainY)
        print("Logistic Regression")
        print(clf.classes_)
        x = clf.predict_proba(testX.reshape(1, -1))
        print(x)
        avCl2 += x[0][0]
        avLu2 += x[0][1]
        avPa2 += x[0][2]
        avPu2 += x[0][3]

        from sklearn.svm import LinearSVC
        clf = LinearSVC()
        clf.fit(trainX, trainY)
        x = clf.predict(testX.reshape(1,-1))
        print("LinearSVC")
        print(x[0])
        if (x[0] == "Ignatius"):
            svcCount += 1
        print("----------------------------")

    print("Averages MLP: ", avCl1/iterations, avLu1/iterations, avPa1/iterations, avPu1/iterations)
    print("Averages LogReg: ", avCl2/iterations, avLu2/iterations, avPa2/iterations, avPu2/iterations)
    print("SVC Accuracy: ", svcCount/iterations)

In [179]:
#whole document
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[6.0537583e-01 4.5478370e-04 1.6547565e-06 3.9416769e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.42887750e-01 4.17878321e-04 4.61745974e-03 1.52076912e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.9401677e-01 5.9832046e-03 1.0826007e-11 1.0224751e-10]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.93617900e-01 1.61049189e-04 1.26498032e-03 1.04956070e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[5.2966923e-01 4.5730476e-06 1.5211183e-01 3.1821442e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.68876435e-01 3.02858571e-04 2.84815079e-03 1.27972555e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[2.9790976e-07 2.0747086e-08 1.1027028e-03 9.9889696e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.00329649e-01 2.51833

In [183]:
#paragraph with n sent = 5
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[3.2914037e-04 9.9935991e-01 2.3489806e-04 7.6062890e-05]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.18357611e-01 1.66321733e-04 2.89066513e-03 7.85854021e-02]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[5.9416944e-01 2.7063194e-07 5.4426626e-11 4.0583029e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.68370835e-01 1.51603578e-04 1.67220178e-03 1.29805360e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[6.7148724e-24 9.2833716e-13 9.9876678e-01 1.2332035e-03]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.16803145e-01 2.34072568e-04 7.15488305e-03 7.58078991e-02]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.1180898e-04 1.2355931e-03 9.3392696e-04 9.9691868e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.10176497e-01 3.36713

In [185]:
#paragraph with n sent = 7
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[4.7311086e-02 9.5266813e-01 6.9537582e-06 1.3868953e-05]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.51043986e-01 4.43812684e-04 3.78706484e-03 1.44725136e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.0608961e-06 5.6579133e-06 4.0165344e-07 9.9998593e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.36275649e-01 3.42251465e-04 6.49224564e-03 1.56889853e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[3.5123000e-07 5.6941058e-06 1.0797552e-07 9.9999380e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.40329582e-01 2.22570190e-04 2.33041619e-03 1.57117431e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.34881765 0.1598119  0.23989627 0.2514742 ]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.92753267e-01 2.73723479e-04 2.63

In [187]:
#paragraph with n sent = 3
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[3.2255519e-04 3.6131478e-05 9.7535747e-01 2.4283823e-02]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.06364931e-01 1.91113129e-04 2.39222040e-03 9.10517357e-02]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.02006731 0.11463881 0.00101844 0.8642754 ]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[9.01241206e-01 3.06695721e-04 4.04075606e-03 9.44113427e-02]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.72897416 0.1256226  0.05269062 0.09271268]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.92449705e-01 3.61636165e-04 3.58100777e-03 1.03607651e-01]]
LinearSVC
Ignatius
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[1.0024326e-06 7.5820039e-06 5.9165664e-08 9.9999130e-01]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[8.41475885e-01 3.92378819e-04 2.67065414e-03 1

In [6]:
#POS tags the documents
from angel import tag
longTags = tag(longFreq)
markTags = tag(markFreq)
mathTags = tag(mathFreq)
ignatTags = tag(ignatFreq)
testTags = tag(testFreq)

Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 15901 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 9366 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood m

In [69]:
#takes in tag list and returns in string format of the tags in the tag list 
def prepareForD2V(tagList):
    cleaned = ""
    line = ""
    for word in tagList:
        cleaned += word[1]
        cleaned += " "
    return cleaned

In [70]:
#prepares the texts for D2V
longD = prepareForD2V(longTags)
markD = prepareForD2V(markTags)
mathD = prepareForD2V(mathTags)
ignatD = prepareForD2V(ignatTags)
testD = prepareForD2V(testTags)

In [71]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
docs = []
docs.append(TaggedDocument(words=longD, tags=["Longus"]))
docs.append(TaggedDocument(words=markD, tags=["Mark"]))
docs.append(TaggedDocument(words=mathD, tags=["Mathew"]))
docs.append(TaggedDocument(words=ignatD, tags=["Ignatius"]))

In [72]:
#pos tags 
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.26160893 0.28003475 0.2479564  0.21039993]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.12587673 0.22041313 0.34465741 0.30905273]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.18939967 0.10164193 0.62335014 0.0856082 ]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.12530777 0.22608643 0.35793496 0.29067084]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.12551    0.09361644 0.11986823 0.6610053 ]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.1283537  0.21935961 0.34459437 0.30769232]]
LinearSVC
Mathew
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.10318421 0.6525743  0.06387405 0.18036744]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.12665695 0.22206504 0.35723737 0.29404064]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 

In [65]:
# simplified pos tags 
runMLAnalysis()

MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.60415614 0.16520602 0.15648213 0.0741557 ]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.10287065 0.26371444 0.23531179 0.39810312]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.07979446 0.160629   0.13615367 0.62342286]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.10013874 0.23915981 0.22498176 0.43571969]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.09078051 0.13511781 0.39902362 0.37507802]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.09646722 0.26474611 0.19996762 0.43881905]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.05218558 0.65826195 0.19202648 0.09752594]]
Logistic Regression
['Ignatius' 'Longus' 'Mark' 'Mathew']
[[0.09597536 0.26879648 0.20798269 0.42724548]]
LinearSVC
Longus
----------------------------
MLP
['Ignatius' 'Longus' 'Mark' 

In [66]:
docVecs = {}
docVecs['Test'] = model.infer_vector(test.split())
model.docvecs.most_similar(positive=[docVecs['Test']])

[('Longus', 0.038748785853385925),
 ('Mark', 0.029405534267425537),
 ('Mathew', -0.0751970112323761),
 ('Ignatius', -0.10529107600450516)]